# Web Server logs Analysis - Nasa

In [56]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import re

spark = (SparkSession
        .builder
        .appName("Nasa")
        .getOrCreate())

In [57]:
_route = "C:/Users/sergio.fuentes/Developer/IntelliJ/Nasa/resources/"
_file = "access.log"
nasa_file = _route + _file

df = (spark.read.format("text")
        .option("header", "true")
        .option("inferSchema", "true")
        .load(nasa_file))
df.show()

+--------------------+
|               value|
+--------------------+
|in24.inetnebr.com...|
|uplherc.upl.com -...|
|uplherc.upl.com -...|
|uplherc.upl.com -...|
|uplherc.upl.com -...|
|ix-esc-ca2-07.ix....|
|uplherc.upl.com -...|
|slppp6.intermind....|
|piweba4y.prodigy....|
|slppp6.intermind....|
|slppp6.intermind....|
|ix-esc-ca2-07.ix....|
|slppp6.intermind....|
|uplherc.upl.com -...|
|133.43.96.45 - - ...|
|kgtyk4.kj.yamagat...|
|kgtyk4.kj.yamagat...|
|d0ucr6.fnal.gov -...|
|ix-esc-ca2-07.ix....|
|d0ucr6.fnal.gov -...|
+--------------------+
only showing top 20 rows



In [58]:
sample_logs = [item['value'] for item in df.collect()]
# sample_logs

In [59]:
host_pattern = r'(^\S+\.[\S+\.]+\S+)\s'
hosts = [re.search(host_pattern, item).group(1)
           if re.search(host_pattern, item)
           else 'no match'
           for item in sample_logs]

In [60]:
ts_pattern = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]'
timestamps = [re.search(ts_pattern, item).group(1)
                if re.search(ts_pattern, item)
                else 'no match'
                for item in sample_logs]

In [61]:
method_uri_protocol_pattern = r'\"(\S+)\s(\S+)\s*(\S*)\"'
method_uri_protocol = [re.search(method_uri_protocol_pattern, item).groups()
                        if re.search(method_uri_protocol_pattern, item)
                        else 'no match'
                        for item in sample_logs]

In [62]:
status_pattern = r'\s(\d{3})\s'
status = [re.search(status_pattern, item).group(1) 
            if re.search(status_pattern, item)
            else 'no match'
            for item in sample_logs]

In [63]:
content_size_pattern = r'\s(\d+|-)$'
content_size = [re.search(content_size_pattern, item).group(1)
            if re.search(content_size_pattern, item)
            else 'no match'
            for item in sample_logs]

In [64]:
from pyspark.sql.functions import regexp_extract

logs_df = df.select(
    regexp_extract('value', host_pattern, 1).alias('host'),
    regexp_extract('value', ts_pattern, 1).alias('timestamp'),
    regexp_extract('value', method_uri_protocol_pattern, 1).alias('method'),
    regexp_extract('value', method_uri_protocol_pattern, 2).alias('endpoint'),
    regexp_extract('value', method_uri_protocol_pattern, 3).alias('protocol'),
    regexp_extract('value', status_pattern, 1).cast('integer').alias('status'),
    regexp_extract('value', content_size_pattern, 1).alias('content_size'))

logs_df.show(n=10, truncate=-1, vertical=False)
print((logs_df.count(), len(logs_df.columns)))

+---------------------------+--------------------------+------+-----------------------------------------------+--------+------+------------+
|host                       |timestamp                 |method|endpoint                                       |protocol|status|content_size|
+---------------------------+--------------------------+------+-----------------------------------------------+--------+------+------------+
|in24.inetnebr.com          |01/Aug/1995:00:00:01 -0400|GET   |/shuttle/missions/sts-68/news/sts-68-mcc-05.txt|HTTP/1.0|200   |1839        |
|uplherc.upl.com            |01/Aug/1995:00:00:07 -0400|GET   |/                                              |HTTP/1.0|304   |0           |
|uplherc.upl.com            |01/Aug/1995:00:00:08 -0400|GET   |/images/ksclogo-medium.gif                     |HTTP/1.0|304   |0           |
|uplherc.upl.com            |01/Aug/1995:00:00:08 -0400|GET   |/images/MOSAIC-logosmall.gif                   |HTTP/1.0|304   |0           |
|uplherc.upl.

In [65]:

logs_df_completo = df.select(
    regexp_extract('value', host_pattern, 1).alias('host'),
    regexp_extract('value', ts_pattern, 1).alias('timestamp'),
    regexp_extract('value', method_uri_protocol_pattern, 1).alias('method'),
    regexp_extract('value', method_uri_protocol_pattern, 2).alias('endpoint'),
    regexp_extract('value', method_uri_protocol_pattern, 3).alias('protocol'),
    regexp_extract('value', status_pattern, 1).cast('integer').alias('status'),
    regexp_extract('value', content_size_pattern, 1).alias('content_size'),
    F.col("value"))

In [66]:
file_parquet = "/tmp/data/parquet/df_parquet_nasa"

(logs_df.write
    .format("parquet")
    .mode("overwrite")
    .option("compression", "snappy")
    .save(file_parquet))

In [67]:
logs_df_parquet = spark.read.format("parquet").load(file_parquet)

¿Cuáles son los distintos protocolos web utilizados? Agrúpalos.

In [68]:

preg1 = (logs_df_parquet
    .groupBy(F.col('Protocol'))
    .count()
    .show())

+---------+-------+
| Protocol|  count|
+---------+-------+
|        a|      1|
|         |   2765|
|HTTP/V1.0|    163|
| HTTP/1.0|1566969|
+---------+-------+



In [69]:
(logs_df_completo
    .filter(~F.col('Protocol')
    .isin('HTTP/*', 'HTTP/V1.0', 'HTTP/1.0', 'STS-69</a><p>'))
    .select("Protocol","value")
    .show())
    # .show(7000, truncate=False))

+--------+--------------------+
|Protocol|               value|
+--------+--------------------+
|        |pipe1.nyc.pipelin...|
|        |columbia.acc.brad...|
|        |columbia.acc.brad...|
|        |columbia.acc.brad...|
|        |columbia.acc.brad...|
|        |columbia.acc.brad...|
|        |columbia.acc.brad...|
|        |cs1-06.leh.ptd.ne...|
|        |cs1-06.leh.ptd.ne...|
|        |cs1-06.leh.ptd.ne...|
|        |cs1-06.leh.ptd.ne...|
|        |cs1-06.leh.ptd.ne...|
|        |cs1-06.leh.ptd.ne...|
|        |cs1-06.leh.ptd.ne...|
|        |cs1-06.leh.ptd.ne...|
|        |cs1-06.leh.ptd.ne...|
|        |cs1-06.leh.ptd.ne...|
|        |cs1-06.leh.ptd.ne...|
|        |cs1-06.leh.ptd.ne...|
|        |cs1-06.leh.ptd.ne...|
+--------+--------------------+
only showing top 20 rows



¿Cuáles son los códigos de estado más comunes en la web? Agrúpalos y ordénalos para ver cuál es el más común.

In [70]:
preg2 = (logs_df_parquet
            .groupBy(F.col('status'))
            .count().orderBy(F.col('count')
            .desc())
            .show())

+------+-------+
|status|  count|
+------+-------+
|   200|1398988|
|   304| 134146|
|   302|  26497|
|   404|  10056|
|   403|    171|
|   501|     27|
|   400|     10|
|   500|      3|
+------+-------+



In [71]:
(logs_df_completo
    .filter(F.col('status')
    .isNull())
    .select("status", "value")
    .show(truncate=False))

+------+-----+
|status|value|
+------+-----+
+------+-----+



¿Y los métodos de petición (verbos) más utilizados?

In [72]:
preg3 = (logs_df_parquet
            .groupBy(F.col('method'))
            .count()
            .orderBy(F.col('count')
            .desc())
            .show())

+---------------+-------+
|         method|  count|
+---------------+-------+
|            GET|1564929|
|           HEAD|   3965|
|               |    891|
|           POST|    111|
|�|t�9ð'À|u|      2|
+---------------+-------+



In [73]:
(logs_df_completo
    .filter(~F.col('method')
    .isin('GET', 'HEAD', 'POST', ''))
    .select("method", "value")
    .show(2000, truncate=False))

+---------------+----------------------------------------------------------------------------+
|method         |value                                                                       |
+---------------+----------------------------------------------------------------------------+
|�|t�9ð'À|u|163.206.42.13 - - [31/Aug/1995:11:04:42 -0400] "�|t�9ð'À|u&�G" 400 -|
|�|t�9ð'À|u|163.206.42.13 - - [31/Aug/1995:11:04:49 -0400] "�|t�9ð'À|u&�G" 400 -|
+---------------+----------------------------------------------------------------------------+



¿Qué recurso tuvo la mayor transferencia de bytes de la página web?

In [74]:
preg4 = (logs_df_parquet
            .groupBy(F.col('endpoint'))
            .agg(F.sum(F.col('content_size'))
            .alias('content_size'))
            .orderBy(F.col('content_size')
            .desc())
            .show(truncate=False))

+------------------------------------------------------------+-------------+
|endpoint                                                    |content_size |
+------------------------------------------------------------+-------------+
|/shuttle/missions/sts-71/movies/sts-71-launch.mpg           |1.639380464E9|
|/shuttle/missions/sts-69/count69.gif                        |1.005927794E9|
|/shuttle/missions/sts-69/movies/sts-69-rollback.mpg         |5.12058235E8 |
|/shuttle/technology/sts-newsref/stsref-toc.html             |4.93211198E8 |
|/shuttle/missions/sts-69/movies/ws-animation-deploy.mpg     |4.64050354E8 |
|/shuttle/missions/sts-53/movies/sts-53-launch.mpg           |4.39216472E8 |
|/shuttle/countdown/video/livevideo2.gif                     |4.07684744E8 |
|/images/rss.gif                                             |3.24585755E8 |
|/images/ksclogo-medium.gif                                  |3.20524106E8 |
|/shuttle/missions/sts-71/movies/sts-71-mir-dock.mpg         |3.0956846E8  |

Además, queremos saber que recurso de nuestra web es el que más tráfico recibe. Es decir, el recurso con más registros en nuestro log.

In [75]:
preg5 = (logs_df_parquet
            .groupBy(F.col('endpoint'))
            .agg(F.count(F.col('endpoint'))
            .alias('count'))
            .orderBy(F.col('count')
            .desc())
            .show(truncate=False))

+-----------------------------------------------+-----+
|endpoint                                       |count|
+-----------------------------------------------+-----+
|/images/NASA-logosmall.gif                     |97384|
|/images/KSC-logosmall.gif                      |75332|
|/images/MOSAIC-logosmall.gif                   |67441|
|/images/USA-logosmall.gif                      |67061|
|/images/WORLD-logosmall.gif                    |66437|
|/images/ksclogo-medium.gif                     |62771|
|/ksc.html                                      |43683|
|/history/apollo/images/apollo-logo1.gif        |37824|
|/images/launch-logo.gif                        |35135|
|/                                              |30328|
|/images/ksclogosmall.gif                       |27808|
|/shuttle/missions/sts-69/mission-sts-69.html   |24606|
|/shuttle/countdown/                            |24458|
|/shuttle/missions/sts-69/count69.gif           |24381|
|/shuttle/missions/sts-69/sts-69-patch-small.gif

¿Qué días la web recibió más tráfico?

In [76]:
logs_df2_parquet = logs_df_parquet.withColumn('dia', F.substring_index(F.col("timestamp"), ":", 1))

preg6 = (logs_df2_parquet
            .groupBy(F.col('dia'))
            .agg(F.count(F.col('endpoint'))
            .alias('count'))
            .orderBy(F.col('count')
            .desc())
            .show(truncate=False))

+-----------+-----+
|dia        |count|
+-----------+-----+
|31/Aug/1995|90125|
|30/Aug/1995|80641|
|29/Aug/1995|67988|
|10/Aug/1995|61248|
|11/Aug/1995|61246|
|09/Aug/1995|60458|
|08/Aug/1995|60157|
|14/Aug/1995|59878|
|04/Aug/1995|59557|
|17/Aug/1995|58988|
|15/Aug/1995|58847|
|23/Aug/1995|58097|
|22/Aug/1995|57762|
|07/Aug/1995|57362|
|25/Aug/1995|57321|
|16/Aug/1995|56653|
|18/Aug/1995|56246|
|21/Aug/1995|55540|
|28/Aug/1995|55496|
|24/Aug/1995|52552|
+-----------+-----+
only showing top 20 rows



¿Cuáles son los hosts son los más frecuentes?

In [77]:
preg7 = (logs_df_parquet
            .groupBy(F.col('host'))
            .count()
            .orderBy(F.col('count')
            .desc())
            .show(truncate=False))

+--------------------+-----+
|host                |count|
+--------------------+-----+
|edams.ksc.nasa.gov  |6530 |
|piweba4y.prodigy.com|4846 |
|163.206.89.4        |4791 |
|piweba5y.prodigy.com|4607 |
|piweba3y.prodigy.com|4416 |
|www-d1.proxy.aol.com|3889 |
|www-b2.proxy.aol.com|3534 |
|www-b3.proxy.aol.com|3463 |
|www-c5.proxy.aol.com|3423 |
|www-b5.proxy.aol.com|3411 |
|www-c2.proxy.aol.com|3407 |
|www-d2.proxy.aol.com|3404 |
|www-a2.proxy.aol.com|3337 |
|news.ti.com         |3298 |
|www-d3.proxy.aol.com|3296 |
|www-b4.proxy.aol.com|3293 |
|www-c3.proxy.aol.com|3272 |
|www-d4.proxy.aol.com|3234 |
|www-c1.proxy.aol.com|3177 |
|www-c4.proxy.aol.com|3134 |
+--------------------+-----+
only showing top 20 rows



In [78]:
(logs_df_completo
    .filter(F.col('host')
    .isin(''))
    .select("host", "value")
    .show())
    # .show(2000, truncate=False))

+----+--------------------+
|host|               value|
+----+--------------------+
|    |triton - - [01/Au...|
|    |triton - - [01/Au...|
|    |triton - - [01/Au...|
|    |triton - - [01/Au...|
|    |triton - - [01/Au...|
|    |triton - - [01/Au...|
|    |brass - - [01/Aug...|
|    |brass - - [01/Aug...|
|    |brass - - [01/Aug...|
|    |brass - - [01/Aug...|
|    |brass - - [01/Aug...|
|    |triton - - [01/Au...|
|    |triton - - [01/Au...|
|    |triton - - [01/Au...|
|    |triton - - [01/Au...|
|    |triton - - [01/Au...|
|    |triton - - [01/Au...|
|    |triton - - [01/Au...|
|    |triton - - [01/Au...|
|    |triton - - [01/Au...|
+----+--------------------+
only showing top 20 rows



¿A qué horas se produce el mayor número de tráfico en la web?

In [79]:
logs_df3_parquet = logs_df_parquet.withColumn('hora', F.substring(F.col("timestamp"), 13, 2))

preg8 = (logs_df3_parquet
            .groupBy(F.col('hora'))
            .agg(F.count(F.col('endpoint'))
            .alias('count'))
            .orderBy(F.col('count')
            .desc())
            .show(truncate=False))

+----+------+
|hora|count |
+----+------+
|15  |109465|
|12  |105143|
|13  |104536|
|14  |101394|
|16  |99527 |
|11  |95344 |
|10  |88309 |
|17  |80834 |
|09  |78695 |
|18  |66809 |
|08  |65443 |
|22  |60673 |
|20  |59944 |
|19  |59315 |
|21  |57985 |
|23  |54570 |
|00  |47862 |
|07  |47386 |
|01  |38531 |
|02  |32508 |
+----+------+
only showing top 20 rows



¿Cuál es el número de errores 404 que ha habido cada día?

In [80]:
preg9 = (logs_df2_parquet
            .where("status == '404'")
            .groupBy(F.col('dia'))
            .count()
            .show(truncate=False))

+-----------+-----+
|dia        |count|
+-----------+-----+
|21/Aug/1995|305  |
|06/Aug/1995|373  |
|07/Aug/1995|537  |
|11/Aug/1995|263  |
|03/Aug/1995|304  |
|18/Aug/1995|256  |
|17/Aug/1995|271  |
|14/Aug/1995|287  |
|20/Aug/1995|312  |
|24/Aug/1995|420  |
|27/Aug/1995|370  |
|13/Aug/1995|216  |
|15/Aug/1995|327  |
|25/Aug/1995|415  |
|22/Aug/1995|288  |
|08/Aug/1995|391  |
|23/Aug/1995|345  |
|19/Aug/1995|209  |
|26/Aug/1995|366  |
|31/Aug/1995|526  |
+-----------+-----+
only showing top 20 rows

